## Batch monitoring

In [ ]:
import pickle
import pandas as pd
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric
from evidently.metric_preset import DataDriftPreset, DataQualityPreset, ClassificationPreset

from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter
from evidently.renderers.html_widgets import WidgetSize

_load saved model_

In [4]:
with open("../notebooks/model.pkl", "rb") as f_out:
    model = pickle.load(f_out)

In [15]:
# use the INCART_Arrhythmia.csv for reference and MIT-BIH Arrhythmia Database.csv to stimulate production data
data = pd.read_csv("../data/INCART_Arrhythmia.csv")
reference_data = data[:1000]
production_data = data[1000:]

In [16]:
reference_data.head()

,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,I01,N,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,I01,N,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,I01,N,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,I01,VEB,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,I01,N,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232


### Evidently AI Metrics

In [7]:
numerical_features = ["record"]
categorical_features = ['0_pre-RR', '0_post-RR', '0_pPeak', '0_tPeak',
       '0_rPeak', '0_sPeak', '0_qPeak', '0_qrs_interval', '0_pq_interval',
       '0_qt_interval', '0_st_interval', '0_qrs_morph0', '0_qrs_morph1',
       '0_qrs_morph2', '0_qrs_morph3', '0_qrs_morph4', '1_pre-RR', '1_post-RR',
       '1_pPeak', '1_tPeak', '1_rPeak', '1_sPeak', '1_qPeak', '1_qrs_interval',
       '1_pq_interval', '1_qt_interval', '1_st_interval', '1_qrs_morph0',
       '1_qrs_morph1', '1_qrs_morph2', '1_qrs_morph3', '1_qrs_morph4']

In [17]:
column_mapping = ColumnMapping(
    target=None,
    prediction='type',
    numerical_features=numerical_features,
    categorical_features=categorical_features
)

In [18]:
report = Report(metrics=[
    ColumnDriftMetric(column_name='type'),
    DatasetDriftMetric(),
    DatasetMissingValuesMetric()
])

In [19]:
reference_data.shape

(1000, 34)

In [ ]:
report.run(reference_data=reference_data, current_data=production_data, column_mapping=column_mapping)

/root/.local/share/virtualenvs/ECG-Arrhythmia-Classifier-fcqqu0k5/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning:

divide by zero encountered in divide

/root/.local/share/virtualenvs/ECG-Arrhythmia-Classifier-fcqqu0k5/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning:

divide by zero encountered in divide

/root/.local/share/virtualenvs/ECG-Arrhythmia-Classifier-fcqqu0k5/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7335: RuntimeWarning:

divide by zero encountered in divide



In [12]:
report.show(mode='inline')

KeyboardInterrupt: 

In [ ]:
result = report.as_dict()

In [ ]:
#prediction drift
result['metrics'][0]['result']['drift_score']

In [ ]:
#share of missing values
result['metrics'][2]['result']['current']['share_of_missing_values']

### Evidently UI and preset

In [ ]:
ws = Workspace("workspace")

In [ ]:
project = ws.create_project("arrythmia classifier")
project.description = "My project description"
project.save()

In [ ]:
regular_report = Report(
    metrics=[
        DataQualityPreset(),
        ClassificationPreset()
    ]
)

regular_report.run(reference_data=None,
                  current_data= reference_data,
                  column_mapping=column_mapping)

regular_report